In [1]:
!pip install websocket-client

     |████████████████████████████████| 71kB 4.6MB/s 


In [2]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib

--2021-05-07 18:51:33--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 216.105.38.13
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|216.105.38.13|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-05-07 18:51:33--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: https://newcontinuum.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-05-07 18:51:33--  https://newcontinuum.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving newcon

In [3]:
import websocket
import json
import numpy as np

In [4]:
cc = 'btcusd'
interval = '1m'

socket = f'wss://stream.binance.com:9443/ws/{cc}t@kline_{interval}'

In [5]:
# Trading Strategy Parameters
aroon_time_period = 5

amount = 1000
core_trade_amount = amount*0.80
core_quantity = 0
trade_amount = amount*0.20
core_to_trade = True

portfolio = 0
investment, real_time_portfolio_value, closes, highs, lows = [], [], [], [], []
money_end = amount

In [6]:
# Buying and Selling functions

def buy(allocated_money, price):
  global portfolio, money_end
  quantity = allocated_money/price
  money_end -= quantity*price
  portfolio += quantity
  if investment == []:
    investment.append(allocated_money)
  else:
    investment.append(allocated_money)
    investment[-1] += investment[-2]

def sell(allocated_money, price):
  global money_end, portfolio
  quantity = allocated_money / price
  money_end += allocated_money
  portfolio -= quantity
  investment.append(-allocated_money)
  investment[-1] += investment[-2]

In [7]:
# Bitcoin Bot

def on_close(ws):
  port_value = portfolio*closes[-1]
  if port_value > 0:
    sell(port_value,price = closes[-1])
  else:
    buy(-port_value, price = closes[-1])
  money_end += investment[-1]
  print('All trades settled')

def on_message(ws,message):
  global portfolio, investment, closes, highs, lows, money_end, core_to_trade, core_quantity, real_time_portfolio_value
  json_message = json.loads(message)
  cs = json_message['k']
  candle_closed, close, high, low = cs['x'], cs['c'], cs['h'], cs['l']

  if candle_closed:
    closes.append(float(close))
    highs.append(float(high))
    lows.append(float(low))
    last_price = closes[-1]
    print(f'Closes: {closes}')
    
    if core_to_trade:
      buy(core_trade_amount, price=closes[-1])
      print(f'Core Investment: We bought ${core_trade_amount} worth of bitcoin', '\n')
      core_quantity += core_trade_amount/closes[-1]
      core_to_trade = False

    aroon = talib.AROONOSC(np.array(highs), np.array(lows), aroon_time_period)
    last_aroon = round(aroon[-1],2)
    amt = last_aroon*trade_amount/100
    port_value = portfolio*last_price - core_quantity*last_price
    trade_amt = amt - port_value
    RT_portfolio_value = money_end + port_value + core_quantity*last_price
    real_time_portfolio_value.append(float(RT_portfolio_value))
    print(f'The Last Aroon is "{last_aroon}" and recommended exposure is "${amt}"')
    print(f'Real-Time Portfolio Value: ${RT_portfolio_value}', '\n')
    
    if trade_amt > 0:
      buy(trade_amt, price=last_price)
      print(f'We bought ${trade_amt} worth of bitcoin', '\n', '\n')
    elif trade_amt < 0:
      sell(-trade_amt, price=last_price)
      print(f'We sold ${-trade_amt} worth of bitcoin', '\n', '\n')

ws = websocket.WebSocketApp(socket, on_message=on_message, on_close=on_close)

In [8]:
ws.run_forever()

Closes: [57890.13]
Core Investment: We bought $800.0 worth of bitcoin 

The Last Aroon is "nan" and recommended exposure is "$nan"
Real-Time Portfolio Value: $1000.0 

Closes: [57890.13, 57893.96]
The Last Aroon is "nan" and recommended exposure is "$nan"
Real-Time Portfolio Value: $1000.0529278479769 

Closes: [57890.13, 57893.96, 57876.36]
The Last Aroon is "nan" and recommended exposure is "$nan"
Real-Time Portfolio Value: $999.8097084943496 

Closes: [57890.13, 57893.96, 57876.36, 57850.34]
The Last Aroon is "nan" and recommended exposure is "$nan"
Real-Time Portfolio Value: $999.4501307908619 

Closes: [57890.13, 57893.96, 57876.36, 57850.34, 57858.03]
The Last Aroon is "nan" and recommended exposure is "$nan"
Real-Time Portfolio Value: $999.5564010652594 

Closes: [57890.13, 57893.96, 57876.36, 57850.34, 57858.03, 57913.56]
The Last Aroon is "-80.0" and recommended exposure is "$-160.0"
Real-Time Portfolio Value: $1000.3237857645163 

We sold $160.0 worth of bitcoin 
 

Closes: [

True

In [1]:
portfolio

NameError: ignored

In [ ]:
investment

[800.0,
 714.28,
 714.3315816565719,
 714.2545733422168,
 671.3815771663475,
 657.007325092993,
 657.1340845698853,
 657.1139244187802,
 657.1140278187621,
 657.0651196627199,
 671.4754740224469,
 685.7783341187313,
 700.08906490421,
 714.2931509583352,
 599.9323328386247,
 785.8070272863039,
 785.8074097315766,
 785.8144615234537,
 785.8070387141614,
 785.8015837038079,
 785.8179291379993,
 900.091628764731,
 1.0644130290421572]

In [ ]:
money_end

998.935586970958

In [ ]:
# Return and Comparison with Benchmark
beg = closes[0]
end = closes[-1]

btc_return = (end/beg-1)
btc_return*100

-0.12159803303889438

In [ ]:
bot_return = (money_end/amount - 1)
bot_return*100

-0.10644130290420328

In [ ]:
alpha = bot_return - btc_return
alpha*100

0.015156730134691099

In [ ]:
btc_risk = np.std(np.log(np.array(closes[1:]) / np.array(closes[:-1])))
btc_risk

0.0006046054692614492

In [ ]:
bot_risk = np.std(np.log(np.array(real_time_portfolio_value[1:]) / np.array(real_time_portfolio_value[:-1])))
bot_risk

0.0004502650557083422

In [ ]:
btc_sharpe_ratio = btc_return / btc_risk
round(btc_sharpe_ratio,3)

-2.011